In [13]:
import requests
from bs4 import BeautifulSoup

class OlympicScraper:
    def __init__(self, year, link,db):
        self.year = year
        self.link = link
         # db.insert_olympic(obj.data)
        self.db=db
        # self.olympics_db=olympics_db
        #    OlympicScraper(year, link,olympics_db)
        self.data = {
            'Year': None, 'Name': None, 'WikipediaURL': link, 'HostCity': None,
            'Athletes': None, 'Rank1': None, 'Rank2': None, 'Rank3': None,
            'ParticipatingNations': [], 'Sports': []
        }
        self.soup = None
        self.find_all()

    def fetch_page(self):
        resp = requests.get(self.link)
        resp.raise_for_status()
        self.soup = BeautifulSoup(resp.content, 'html.parser')
    
    def fetch_name(self):
        h1 = self.soup.find('h1', id='firstHeading')
        self.data['Name'] = h1.get_text(strip=True)
    
    def fetch_year(self):
        if not self.data['Year']:
            if self.data['Name']:
                self.data['Year'] = self.data['Name'].split()[0]
            else:
                self.data['Year'] = self.year
    
    def fetch_hostcity(self):
        info = self.soup.find('table', class_='infobox')
        for tr in info.find_all('tr'):
            th = tr.find('th')
            if th and th.get_text(strip=True).lower() in ['location', 'host city']:
                td = tr.find('td')
                if td:
                    self.data['HostCity'] = td.get_text(strip=True)
                    return

    def fetch_athletes(self):
        info = self.soup.find('table', class_='infobox')
        for tr in info.find_all('tr'):
            th = tr.find('th')
            if th and 'athletes' in th.get_text(strip=True).lower():
                td = tr.find('td')
                if td:
                    val = td.get_text(strip=True).split('(')[0].strip()
                    self.data['Athletes'] = val
                    return

    def fetch_participating_nations(self):
        tables = self.soup.find_all('table', class_=['wikitable', 'collapsible', 'mw-collapsible', 'mw-made-collapsible'])
        for table in tables:
            th = table.find('th')
            if th and 'participating' in th.get_text(strip=True).lower():
                lis = [li.get_text(strip=True).split('(')[0].strip() for li in table.find_all('li')]
                self.data['ParticipatingNations'] = lis
                return
    
    def fetch_sports(self):
        tables = self.soup.find_all('table', class_=['wikitable', 'collapsible', 'mw-collapsible', 'mw-made-collapsible'])
        for table in tables:
            th = table.find('th')
            if th and 'summer olympic sports programme' in th.get_text(strip=True).lower():
                lis = [li.get_text(strip=True).split('(')[0].strip() for li in table.find_all('li')]
                self.data['Sports'] = lis
                return

    def fetch_rankings(self):
        tables = self.soup.find_all('table', class_=['wikitable', 'collapsible', 'mw-collapsible', 'mw-made-collapsible'])
        for table in tables:
            caption = table.find('caption')
            if caption and 'summer olympics medal table' in caption.get_text(strip=True).lower():
                rows = table.find('tbody').find_all('tr')
                for idx, row in enumerate(rows[1:4], 1):  # First 3 ranks only
                    th = row.find('th')
                    if th:
                        self.data[f'Rank{idx}'] = th.get_text(strip=True)
                return

    def find_all(self):
        self.fetch_page()
        self.fetch_name()
        self.fetch_year()
        self.fetch_hostcity()
        self.fetch_athletes()
        self.fetch_participating_nations()
        self.fetch_sports()
        self.fetch_rankings()
        self.db.insert_olympic(self.data)
        # self.db=db
        # self.olympics_db.insert_olympic(self.data)
        
        # return self.data




In [14]:
# import sqlite3

# class OlympicsDatabase:
#     def __init__(self, db_path='olympics.db'):
#         self.conn = sqlite3.connect(db_path)
#         self.create_tables()
    
#     def create_tables(self):
#         cur = self.conn.cursor()
#         # Parent
#         cur.execute("""
#             CREATE TABLE IF NOT EXISTS SummerOlympics (
#                 Year TEXT PRIMARY KEY,
#                 Name TEXT,
#                 WikipediaURL TEXT,
#                 HostCity TEXT,
#                 Athletes TEXT,
#                 Rank1 TEXT,
#                 Rank2 TEXT,
#                 Rank3 TEXT
#             )
#         """)
#         # Child 1
#         cur.execute("""
#             CREATE TABLE IF NOT EXISTS ParticipatingNations (
#                 id INTEGER PRIMARY KEY AUTOINCREMENT,
#                 Year TEXT,
#                 Nation TEXT,
#                 FOREIGN KEY (Year) REFERENCES SummerOlympics(Year) ON DELETE CASCADE
#             )
#         """)
#         # Child 2
#         cur.execute("""
#             CREATE TABLE IF NOT EXISTS Sports (
#                 id INTEGER PRIMARY KEY AUTOINCREMENT,
#                 Year TEXT,
#                 Sport TEXT,
#                 FOREIGN KEY (Year) REFERENCES SummerOlympics(Year) ON DELETE CASCADE
#             )
#         """)
#         self.conn.commit()

#     def insert_olympic(self, olympic_data):
#         cur = self.conn.cursor()
#         # Insert Parent
#         try:
#             cur.execute("""
#                 INSERT INTO SummerOlympics (Year, Name, WikipediaURL, HostCity, Athletes, Rank1, Rank2, Rank3)
#                 VALUES (?, ?, ?, ?, ?, ?, ?, ?)
#             """, (
#                 olympic_data['Year'], olympic_data['Name'], olympic_data['WikipediaURL'],
#                 olympic_data['HostCity'], olympic_data['Athletes'],
#                 olympic_data.get('Rank1'), olympic_data.get('Rank2'), olympic_data.get('Rank3')
#             ))
#         except sqlite3.IntegrityError:
#             print(f"Year {olympic_data['Year']} already in database. Skipping.")
#             return
#         # Insert Child1: ParticipatingNations
#         for nation in olympic_data.get('ParticipatingNations', []):
#             cur.execute("""
#                 INSERT INTO ParticipatingNations (Year, Nation) VALUES (?, ?)
#             """, (olympic_data['Year'], nation))
#         # Insert Child2: Sports
#         for sport in olympic_data.get('Sports', []):
#             cur.execute("""
#                 INSERT INTO Sports (Year, Sport) VALUES (?, ?)
#             """, (olympic_data['Year'], sport))
#         self.conn.commit()
#         print(f"Inserted Olympic data for Year {olympic_data['Year']}")

#     def close(self):
#         self.conn.close()


import sqlite3

class OlympicsDatabase:
    def __init__(self, db_path='olympics.db'):
        self.db_path = db_path
        self._init_schema()  # Run once to create tables
    
    def _init_schema(self):
        # This part runs once in the main thread to create tables
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()

        cur.execute("""
            CREATE TABLE IF NOT EXISTS SummerOlympics (
                Year TEXT PRIMARY KEY,
                Name TEXT,
                WikipediaURL TEXT,
                HostCity TEXT,
                Athletes TEXT,
                Rank1 TEXT,
                Rank2 TEXT,
                Rank3 TEXT
            )
        """)
        cur.execute("""
            CREATE TABLE IF NOT EXISTS ParticipatingNations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Year TEXT,
                Nation TEXT,
                FOREIGN KEY (Year) REFERENCES SummerOlympics(Year) ON DELETE CASCADE
            )
        """)
        cur.execute("""
            CREATE TABLE IF NOT EXISTS Sports (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Year TEXT,
                Sport TEXT,
                FOREIGN KEY (Year) REFERENCES SummerOlympics(Year) ON DELETE CASCADE
            )
        """)
        conn.commit()
        conn.close()

    def insert_olympic(self, olympic_data):
        # Create a fresh connection inside the thread
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()

        try:
            cur.execute("""
                INSERT INTO SummerOlympics (Year, Name, WikipediaURL, HostCity, Athletes, Rank1, Rank2, Rank3)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                olympic_data['Year'], olympic_data['Name'], olympic_data['WikipediaURL'],
                olympic_data['HostCity'], olympic_data['Athletes'],
                olympic_data.get('Rank1'), olympic_data.get('Rank2'), olympic_data.get('Rank3')
            ))
        except sqlite3.IntegrityError:
            print(f"Year {olympic_data['Year']} already in database. Skipping.")
            conn.close()
            return

        for nation in olympic_data.get('ParticipatingNations', []):
            cur.execute("""
                INSERT INTO ParticipatingNations (Year, Nation) VALUES (?, ?)
            """, (olympic_data['Year'], nation))

        for sport in olympic_data.get('Sports', []):
            cur.execute("""
                INSERT INTO Sports (Year, Sport) VALUES (?, ?)
            """, (olympic_data['Year'], sport))

        conn.commit()
        conn.close()
        print(f"Inserted Olympic data for Year {olympic_data['Year']}")


In [8]:

# import pandas as pd
# df = pd.read_excel('summer_olympics_links.xlsx')
# k = 2  # or any value
# sampled_df = df.sample(n=k, random_state=23)
# sampled_links = dict(zip(sampled_df['Year'], sampled_df['Link']))

# # Initialize DB
# olympics_db = OlympicsDatabase('olympics.db')

# # Scrape and insert
# for year, link in sampled_links.items():
#     scraper = OlympicScraper(year, link)
#     olympic_data = scraper.find_all()
#     olympics_db.insert_olympic(olympic_data)

# olympics_db.close()
# import threading
# from queue import Queue
# import pandas as pd

# df = pd.read_excel('summer_olympics_links.xlsx')
# sampled_links = dict(zip(df['Year'], df['Link']))
# for year, link in sampled_links.items():
#     scraper = OlympicScraper(year, link)
#     olympic_data = scraper.data()
#     olympics_db.insert_olympic(olympic_data)
    
# print('done')
# 

In [21]:


import threading
from queue import Queue
import pandas as pd

import time

# Start timer
start_time = time.time()
# Read full Excel file
# Read the links from Excel
df = pd.read_excel('summer_olympics_links.xlsx')
sampled_links = dict(zip(df['Year'], df['Link']))

# Initialize DB
olympics_db = OlympicsDatabase('olympicsmain.db')

# Create a task queue and fill it
task_queue = Queue()
for year, link in sampled_links.items():
    task_queue.put((year, link))

# Thread management
threads = []
created_objects = []
lock = threading.Lock()
NUM_WORKERS = 3  # Number of parallel threads
# there are 4 core in my Cpu so I will use 3 threads
# Worker function
def createObjects(q, db):
    while True:
        try:
            year, link = q.get_nowait()
        except Exception:  # Explicit exception type can be 'queue.Empty'
            break
        obj = OlympicScraper(year, link,db)

        # Insert into database
        # db.insert_olympic(obj.data)

        # Store the object safely
        with lock:
            created_objects.append(obj)

        q.task_done()

# Create and start threads
for _ in range(NUM_WORKERS):
    t = threading.Thread(target=createObjects, args=(task_queue, olympics_db))
    t.start()
    threads.append(t)

# Wait for all threads to finish
for t in threads:
    t.join()

# (Optional) created_objects now contains all the scraper instances
end_time = time.time()
elapsed_time = end_time - start_time

print(f"All data inserted successfully! Time taken: {elapsed_time:.2f} seconds")


Inserted Olympic data for Year 1904
Inserted Olympic data for Year 1896
Inserted Olympic data for Year 1900
Inserted Olympic data for Year 1908
Inserted Olympic data for Year 1920
Inserted Olympic data for Year 1912
Inserted Olympic data for Year 1928
Inserted Olympic data for Year 1924
Inserted Olympic data for Year 1932
Inserted Olympic data for Year 1936
Inserted Olympic data for Year 1948
Inserted Olympic data for Year 1952
Inserted Olympic data for Year 1956
Inserted Olympic data for Year 1964
Inserted Olympic data for Year 1960
Inserted Olympic data for Year 1972
Inserted Olympic data for Year 1976
Inserted Olympic data for Year 1968
Inserted Olympic data for Year 1980
Inserted Olympic data for Year 1984
Inserted Olympic data for Year 1988
Inserted Olympic data for Year 1992
Inserted Olympic data for Year 1996
Inserted Olympic data for Year 2000
Inserted Olympic data for Year 2008
Inserted Olympic data for Year 2004
Inserted Olympic data for Year 2012
Inserted Olympic data for Ye

In [19]:
import sqlite3
# olympics_db = OlympicsDatabase('olympicsmain.db')
def print_olympic_tables(db_path='olympicsmain.db'):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    print("\n--- SummerOlympics ---")
    cursor.execute("SELECT * FROM SummerOlympics")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    
    # print("\n--- ParticipatingNations ---")
    # cursor.execute("SELECT * FROM ParticipatingNations")
    # rows = cursor.fetchall()
    # for row in rows:
    #     print(row)
    
    # print("\n--- Sports ---")
    # cursor.execute("SELECT * FROM Sports")
    # rows = cursor.fetchall()
    # for row in rows:
    #     print(row)
    
    conn.close()

print_olympic_tables('olympicsmain.db')  # Pass correct path if needed



--- SummerOlympics ---
('1904', '1904 Summer Olympics', 'https://en.wikipedia.org/wiki/1904_Summer_Olympics', 'St. Louis, United States', '648', None, None, None)
('1920', '1920 Summer Olympics', 'https://en.wikipedia.org/wiki/1920_Summer_Olympics', 'Antwerp, Belgium', '2,626', None, None, None)
('1896', '1896 Summer Olympics', 'https://en.wikipedia.org/wiki/1896_Summer_Olympics', 'Athens, Greece', '241', 'United States', 'Greece*', 'Germany')
('1900', '1900 Summer Olympics', 'https://en.wikipedia.org/wiki/1900_Summer_Olympics', 'Paris, France', '1226[note1]', None, None, None)
('1912', '1912 Summer Olympics', 'https://en.wikipedia.org/wiki/1912_Summer_Olympics', 'Stockholm, Sweden', '2,406', None, None, None)
('1908', '1908 Summer Olympics', 'https://en.wikipedia.org/wiki/1908_Summer_Olympics', 'London, England51°30′49″N0°13′39″W\ufeff / \ufeff51.51362°N 0.22740°W\ufeff /51.51362; -0.22740', '2,008', None, None, None)
('1924', '1924 Summer Olympics', 'https://en.wikipedia.org/wiki/19

In [20]:
import os
import multiprocessing

# Method 1: Using os
print("CPU cores (logical):", os.cpu_count())

# Method 2: Using multiprocessing
print("CPU cores (logical):", multiprocessing.cpu_count())


CPU cores (logical): 4
CPU cores (logical): 4
